In [1]:
import os
from collections import OrderedDict

import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr

% matplotlib inline

## Objective: To save a munged GDP to munged_data/un_gdp_constant_mkt

In [29]:
# Open files
# NY.GDP.PCAP.PP.KD is WDI indicator
'''
NY = national accounts: income
MKTP = market prices
PCAP = per capita
PP = purchasing power (no PP means not PP)
KD = constant (vs CD = current)
'''
# df_gdp is gdp market constant
df_gdp_cst_mkt = pd.read_csv('./gdp/gdp_constant/API_NY.GDP.MKTP.KD_DS2_en_csv_v2.csv', skiprows=4)
meta_data = pd.read_csv('./gdp/gdp_constant/Metadata_Country_API_NY.GDP.MKTP.KD_DS2_en_csv_v2.csv')

In [30]:
df_gdp_cst_mkt.head(3) #checking 

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Aruba,ABW,GDP (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,GDP (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.021272e+10,2.066392e+10,NaN,NaN
2,Angola,AGO,GDP (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039199e+11,1.039199e+11,NaN,NaN


## Convenience functions ported from WB analysis**

In [14]:
def filter_non_countries(_df, _metadata):
    '''
    _df : pd.DataFrame
        either income or population data
        
    _metadata : pd.DataFrame
        metadata on a list of entries including countries and non-countries 
        data source is from the World Bank
        has IncomeGroup column that is not null for countries (217)
    '''
    # Country Code in both dataframes
    _merged = _df.merge(_metadata, on='Country Code')
    # non-countries have no IncomeGroup
    non_country_mask = _merged['IncomeGroup'].isnull()
    merged_country_only = _merged[~non_country_mask]
    return merged_country_only

In [26]:
def select_relevant_cols(_df):
    # returns df with 'Country Name', 'Country Code' and all years (1960 to 2016)
    _cols = ['Country Name'] + ['Country Code'] + [str(yr) for yr in range(1960, 2017)]
    return _df[_cols]

## Look at meta data

In [22]:
meta_data['IncomeGroup'].unique()

array(['High income', 'Low income', 'Lower middle income',
       'Upper middle income', nan], dtype=object)

In [23]:
meta_data['Country Code'].nunique()

263

In [28]:
meta_data['IncomeGroup'].isnull().value_counts() # 217 False (countries) # 46 nan

False    217
True      46
Name: IncomeGroup, dtype: int64

## 1. Filter aggregates (not countries)

In [33]:
df_gdp_countries = filter_non_countries(df_gdp_cst_mkt, meta_data)
df_gdp_countries

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,Unnamed: 62,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
0,Aruba,ABW,GDP (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Latin America & Caribbean,High income,SNA data for 2000-2011 are updated from offici...,Aruba,NaN
1,Afghanistan,AFG,GDP (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.999032e+10,2.021272e+10,2.066392e+10,NaN,NaN,South Asia,Low income,Fiscal year end: March 20; reporting period fo...,Afghanistan,NaN
2,Angola,AGO,GDP (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.008863e+11,1.039199e+11,1.039199e+11,NaN,NaN,Sub-Saharan Africa,Lower middle income,NaN,Angola,NaN
3,Albania,ALB,GDP (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.276823e+10,1.309893e+10,1.355215e+10,NaN,NaN,Europe & Central Asia,Upper middle income,NaN,Albania,NaN
4,Andorra,AND,GDP (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Europe & Central Asia,High income,WB-3 code changed from ADO to AND to align wit...,Andorra,NaN
6,United Arab Emirates,ARE,GDP (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.540756e+11,3.676327e+11,3.787957e+11,NaN,NaN,Middle East & North Africa,High income,NaN,United Arab Emirates,NaN
7,Argentina,ARG,GDP (constant 2010 US$),NY.GDP.MKTP.KD,1.155739e+11,1.218470e+11,1.208089e+11,1.143961e+11,1.259848e+11,1.393006e+11,...,4.437071e+11,4.554532e+11,4.449861e+11,NaN,NaN,Latin America & Caribbean,Upper middle income,National Institute of Statistics and Census re...,Argentina,NaN
8,Armenia,ARM,GDP (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.112310e+10,1.145679e+10,1.147971e+10,NaN,NaN,Europe & Central Asia,Lower middle income,NaN,Armenia,NaN
9,American Samoa,ASM,GDP (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.421674e+08,5.480881e+08,NaN,NaN,NaN,East Asia & Pacific,Upper middle income,NaN,American Samoa,NaN
10,Antigua and Barbuda,ATG,GDP (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.222423e+09,1.272872e+09,1.328569e+09,NaN,NaN,Latin America & Caribbean,High income,NaN,Antigua and Barbuda,NaN
